In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
# import sqlite3
# import mysql.connector as cnt
# import plotly.express as px
import requests

In [8]:
url = 'https://api.meteo.lt/v1/stations'
pg = requests.get(url)
r = pg.json()
stotys = {}
for i in r:
    stotys[i['name']] = i['code']
# print(stotys)
n = list(stotys.keys())
for i, k in enumerate(n):
    print(f'{i}: {k}')



0: Akmenės AMS
1: Alytaus AMS
2: Anykščių AMS
3: Birštono AMS
4: Biržų AMS
5: Dotnuvos AMS
6: Druskininkų AMS
7: Dūkšto AMS
8: Elektrėnų AMS
9: Jonavos AMS
10: Joniškio AMS
11: Jurbarko AMS
12: Kaišiadorių AMS
13: Kalvarijos AMS
14: Kauno AMS
15: Kazlų Rūdos AMS
16: Kelmės AMS
17: Klaipėdos AMS
18: Kretingos AMS
19: Kupiškio AMS
20: Kybartų AMS
21: Laukuvos AMS
22: Lazdijų AMS
23: Marijampolės AMS
24: Mažeikių AMS
25: Molėtų AMS
26: Nidos AMS
27: Pagėgių AMS
28: Pakruojo AMS
29: Panevėžio AMS
30: Plungės AMS
31: Prienų AMS
32: Raseinių AMS
33: Rietavo AMS
34: Rokiškio AMS
35: Šakių AMS
36: Šalčininkų AMS
37: Šeduvos AMS
38: Šiaulių AMS
39: Šilutės AMS
40: Skuodo AMS
41: Švenčionių AMS
42: Tauragės AMS
43: Telšių AMS
44: Trakų AMS
45: Ukmergės AMS
46: Utenos AMS
47: Varėnos AMS
48: Ventės AMS
49: Vėžaičių AMS
50: Vilniaus AMS
51: Zarasų AMS


In [11]:
stations = input('Įveskite stoties numerį(-ius) (iki trijų numerių, atskirtų kableliais)')
st_idx = list(map(int, stations.split(',')))
print('Jūs pasirinkote šias stotis:')
sel_sts = [n[i] for i in st_idx]
print(sel_sts)

Jūs pasirinkote šias stotis:
['Alytaus AMS', 'Anykščių AMS', 'Birštono AMS']
